In [1]:
import numpy as np
import nibabel as nib 
import os 
import pandas as pd
from utils.utils import * 
import math

In [2]:
rois_long = {'V1v': 1, 'V1d': 2, 'V2v': 3, 'V2d': 4, 'V3v': 5, 'V3d': 6, 'hV4': 7, 'VO-1': 8,
 'VO-2': 9, 'PHC-1': 10, 'PHC-2': 11, 'LO-1': 12, 'LO-2': 13, 'TO-1': 14, 'TO-2': 15
}
sior_long = {v: k for k,v in rois_long.items()}
to_reduce = {'V1v': 'V1', 'V1d': 'V1', 'V2v': 'V2', 'V2d': 'V2', 'V3v': 'V3', 'V3d': 'V3'}
sior = {v: k for k,v in rois.items()}

In [3]:
models_subs = {}
models = ['oself']
columns = ["x0", "y0", "sigma", "slope", "intercept",  "var_explained", "mds_ecc", "mds_ang", "roi", "fit_with"]
model_study = 'oself'

for sub, n_sessions in subjects_sessions.values():
    models_subs[sub] = {}
    for m in models:
        m_file = os.path.join(proj_dir, 'models', sub, f'best_fits_{m}_{sub}_{n_sessions}_NR.npy')
        models_subs[sub][m] = pd.DataFrame(np.load(m_file, allow_pickle=True), columns=columns)
        models_subs[sub][m][columns[:-2]] = models_subs[sub][m][columns[:-2]].astype(np.float32)
    sub_m_study = models_subs[sub][model_study]

hier_index = pd.MultiIndex.from_arrays(
    [
        np.tile(list(rois.keys()), len(models_subs)),
        np.repeat(list(models_subs.keys()), len(roi.keys())),
    ],
    names =["roi", "sub"]
)
models_subs

FileNotFoundError: [Errno 2] No such file or directory: '/home/stan/thesis-repo/projects/NSD/models/subj02/best_fits_oself_subj02_40_NR.npy'

In [4]:
models_subs

{'subj01': {'oself':              x0        y0     sigma      slope  intercept  var_explained  \
  0      0.380577 -0.684969  6.695032  28.010561 -27.825483       0.033850   
  1      0.440444 -0.646169  6.927041  25.040047 -24.872763       0.030827   
  2      0.557979 -0.588672  6.817106  18.652100 -18.500206       0.027525   
  3      0.358346  0.325810  0.010035  12.314314  -0.000359       0.000618   
  4      0.365889 -0.626864  5.443463  17.316534 -17.147028       0.028198   
  ...         ...       ...       ...        ...        ...            ...   
  33792 -0.252317  1.050000  7.850662  81.860420 -81.519997       0.071867   
  33793 -0.174751 -0.953210  3.659806  10.413570 -10.227592       0.019183   
  33794 -0.346183  0.436767  6.513120  27.577259 -27.391813       0.027816   
  33795 -0.314958 -0.909629  3.327075   7.510156  -7.325987       0.026303   
  33796 -0.368551 -0.050746  5.348899  16.697830 -16.527172       0.025332   
  
          mds_ecc   mds_ang roi fit_with  

In [5]:
models = ['oself']
hemis = ['lh', 'rh']
fields = columns

for model in models:
    for sub, sessions in subjects_sessions.values():
        m = models_subs[sub][model]
        for f in fields:
            print(f'\t{f}')
            s = -1
            for hemi in hemis:
                print(f'\t\t{hemi}')
                maskdata_long_file = os.path.join(data_dir, 'mask', sub, f'{hemi}.{sub}.testrois.mgz')
                maskdata_long = nib.load(maskdata_long_file).get_fdata().squeeze()
                maskdata_file = os.path.join(data_dir, 'mask', sub, f'{hemi}.short.reduced.{sub}.testrois.npy')
                maskdata = np.load(maskdata_file, allow_pickle=True).astype(int)

                data_out_file = os.path.join(proj_dir, 'results_out', sub, f'{hemi}.{model}.{f}.mgz')
                if os.path.exists(data_out_file):
                    continue
                data_out = np.zeros(maskdata_long.shape)
                wrongs = []
                for i in range(maskdata_long.shape[0]):
                    voxel = int(maskdata_long[i])
                    if voxel==0 or voxel > 15:
                        continue
                    s += 1
                    if sior_long[voxel] in to_reduce.keys():
                        roi = rois[to_reduce[sior_long[voxel]]]
                    else:
                        roi = rois[sior_long[voxel]]
                  #  roi = rois[sior[voxel]]
                    # prove that the order is maintained
                    if roi != rois[m.iloc[s].roi]:
                        print(f'{i}: {roi} != {rois[m.iloc[s].roi]}')
                        wrongs.append(i)
                    if f in ['roi', 'fit_with']:
                        data_out[i] = rois[m.iloc[s][f]]
                    elif f == 'mds_ang':
                        data_out[i] = math.degrees(m.iloc[s][f]) + 180
                    elif f == 'intercept':
                        data_out[i] = m.iloc[s][f] + abs(m[f].min()) + 1e-08
                    elif f in ['x0', 'y0']:
                        data_out[i] = m.iloc[s][f] + 1.1
                    else:
                        data_out[i] = m.iloc[s][f]
                img = nib.Nifti1Image(np.expand_dims(data_out, axis=(1, 2)), np.eye(4))

                nib.loadsave.save(img, data_out_file)
                if f == 'mds_ang':
                    for ang in [45, 90, 135, 180, 225, 270, 305]:
                        data_out_file = os.path.join(proj_dir, 'results_out', sub, f'{hemi}.{model}.f.{str(ang)}.mgz')
                        data_out_ang = np.copy(data_out)
                        data_out_ang[data_out != 0] += ang
                        data_out_ang = data_out_ang % 360
                        img = nib.Nifti1Image(np.expand_dims(data_out_ang, axis = (1, 2)), np.eye(4))
                        nib.loadsave.save(img, data_out_file)


print('end')
                        
            

	x0
		lh


/home/stan/miniconda3/envs/thesis/lib/python3.10/site-packages/nibabel-5.1.0-py3.10.egg/nibabel/spatialimages.py:558: UserWarning: Using large vector Freesurfer hack; header will not be compatible with SPM or FSL
  hdr.set_data_shape(shape)


		rh
	y0
		lh
		rh
	sigma
		lh
		rh
	slope
		lh
		rh
	intercept
		lh
		rh
	var_explained
		lh
		rh
	mds_ecc
		lh
		rh
	mds_ang
		lh
		rh
	roi
		lh
		rh
	fit_with
		lh
		rh


KeyError: 'oself'

In [ ]:
### m = models_subs['subj01']['oself']
m

In [46]:
maskdata_long_file = os.path.join(data_dir,'mask', 'subj01', 'lh.short.reduced.subj01.testrois.npy')
#maskdata_long = nib.load(maskdata_long_file).get_fdata().squeeze()
maskdata_long = np.load(maskdata_long_file, allow_pickle=True)

In [42]:
print(maskdata_long.shape)

(17901,)


In [43]:
maskdata_long_file = os.path.join(data_dir,'mask', 'subj01', 'rh.short.reduced.subj01.testrois.npy')
#maskdata_long = nib.load(maskdata_long_file).get_fdata().squeeze()
maskdata_long = np.load(maskdata_long_file, allow_pickle=True)

In [44]:
print(maskdata_long.shape)

(15896,)


In [52]:
mask1 = os.path.join(data_dir, 'mask', 'subj01', 'short.reduced.subj01.testrois.npy')
mask1 = np.load(mask1, allow_pickle=True)
mask1

array([1., 1., 1., ..., 2., 2., 2.])

In [63]:
np.unique(mask1)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.])